# Clasificador de Flores (128×128, 5 clases)

---

## 1) Comprobar el intérprete de Python

**Código asociado:** `import sys; print(sys.executable)`

**Qué hace / por qué:**

* Muestra la ruta del intérprete de Python que está ejecutando el Notebook (por ejemplo `/home/user/venv/bin/python`).
* Útil para confirmar que estás usando el entorno virtual o conda correcto (evita confusiones con versiones y dependencias).
* Si ves una ruta distinta a la esperada, activa el entorno correcto antes de ejecutar el notebook.

In [ ]:
import sys
print(sys.executable)



## 2) Importar TensorFlow y constantes globales

**Código asociado:** `import tensorflow as tf` y definición de `IMAGE_SIZE`, `BATCH_SIZE`, `DATA_DIR`

**Qué hace / por qué:**

* `import tensorflow as tf` carga TensorFlow/Keras.
* `IMAGE_SIZE = (128,128)` define el tamaño al que se redimensionarán todas las imágenes (consistencia para la red).
* `BATCH_SIZE = 32` controla cuántas imágenes procesa el modelo a la vez (balance entre velocidad y memoria).
* `DATA_DIR = "flores"` es la carpeta raíz con subcarpetas `train/`, `val/`, `test/`.

---


## 3) Carga del dataset con `image_dataset_from_directory`

**Código asociado:** llamadas a `tf.keras.utils.image_dataset_from_directory(...)`

**Qué hace / por qué:**

* Crea `tf.data.Dataset` para `train_ds`, `val_ds`, `test_ds` desde carpetas organizadas por clase (`DATA_DIR/train/<clase>`).
* Parámetros clave:

  * `labels='inferred'`: se infieren las etiquetas a partir de los nombres de las subcarpetas.
  * `label_mode='int'`: las etiquetas serán enteros (`0..num_classes-1`). **Usamos luego `sparse_categorical_crossentropy`.**
  * `image_size=IMAGE_SIZE`: redimensiona las imágenes a 128×128.
  * `batch_size=BATCH_SIZE` y `shuffle=True` (solo en entrenamiento).
  * `seed=123` para reproducibilidad en la selección aleatoria.
* **Salida por lote**:

  * `images` → tensor `(batch_size, 128, 128, 3)`
  * `labels` → tensor `(batch_size,)` (enteros)
* `class_names = train_ds.class_names` lista las etiquetas ordenadas (ej. `['rosa','tulipan',...]`) y `num_classes = len(class_names)`.

---

In [ ]:
import tensorflow as tf

IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32
DATA_DIR = "flores"  # con subcarpetas train, val, test

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR + '/train',
    labels='inferred',
    label_mode='int',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=123
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR + '/val',
    labels='inferred',
    label_mode='int',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR + '/test',
    labels='inferred',
    label_mode='int',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Clases:", class_names)


## 4) Normalización / rendimiento: `cache()` y `prefetch()`

**Código asociado:** `train_ds = train_ds.cache().prefetch(tf.data.AUTOTUNE)` (y para val/test)

**Qué hace / por qué:**

* `cache()`: guarda en memoria (o en disco si se pasa un filename) los datos procesados la primera vez para acelerar épocas posteriores. **Cuidado:** si tu dataset no cabe en RAM, quita `cache()` o usa `cache('ruta_cache')`.
* `prefetch(buffer_size=AUTOTUNE)`: prepara el siguiente lote mientras la GPU/CPU entrena el actual — reduce el tiempo de espera entre batches.
* `AUTOTUNE` deja que TF elija el mejor comportamiento automático.

---

In [ ]:

# Normalización
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds   = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds  = test_ds.cache().prefetch(buffer_size=AUTOTUNE)


## 5) Aumento de datos (Data Augmentation)

**Código asociado:** `data_augmentation = tf.keras.Sequential([...layers.RandomFlip..., ...])`

**Qué hace / por qué:**

* Aplica transformaciones aleatorias a las imágenes durante el entrenamiento: flip, rotación, zoom, cambio de contraste, etc.
* Mejora la capacidad de generalización del modelo al simular variaciones naturales (orientación, zoom, iluminación).
* Estas capas implementadas como `tf.keras.layers` se ejecutan en la GPU y se aplican **solo en modo entrenamiento** cuando están dentro del modelo.
* Ventaja: evita tener que generar y almacenar versiones aumentadas manualmente.

---


In [ ]:
# Data augmentation

from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.15),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
], name="data_augmentation")

## 6) Construcción del modelo CNN (visión general)

**Código asociado:** la función `build_cnn(...)` y `model = build_cnn(...)`

**Arquitectura (resumen):**

1. `Input(shape=(128,128,3))`
2. `Rescaling(1./255)` — normaliza pixeles a `[0,1]`
3. `data_augmentation(...)` — capas de aumento
4. **Bloque Conv 1**: `Conv2D(32, 3x3, relu, padding='same')` → `BatchNormalization()` → `MaxPooling2D(2x2)`
5. **Bloque Conv 2**: `Conv2D(64, 3x3, relu)` → `BatchNormalization()` → `MaxPooling2D`
6. **Bloque Conv 3**: `Conv2D(128, 3x3, relu)` → `BatchNormalization()` → `MaxPooling2D`
7. `GlobalAveragePooling2D()` → `Dropout(dropout_rate)` → `Dense(128, relu)` → `Dropout(0.3)` → `Dense(num_classes, softmax)`

**Por qué esta estructura:**

* Las capas convolucionales extraen características locales (bordes → texturas → partes de objetos).
* `BatchNormalization` estabiliza y acelera el entrenamiento normalizando las activaciones por mini-batch.
* `MaxPooling` reduce resolución espacial (reduce parámetros, agrega invarianza a pequeñas traslaciones).
* `GlobalAveragePooling2D` resume cada mapa de características en un valor promedio (reduce el número de parámetros comparado con `Flatten` + Dense grande).
* `Dropout` es una técnica de regularización para evitar sobreajuste.
* `Dense(..., softmax)` en la salida da probabilidades por clase.

---


In [ ]:
## Construcción del modelo CNN desde cero

from tensorflow.keras import layers, models

def build_cnn(input_shape=(128,128,3), num_classes=5, dropout_rate=0.5):
    inputs = layers.Input(shape=input_shape)

    # Normalización simple (0-255 -> 0-1)
    x = layers.Rescaling(1./255)(inputs)

    # Aumento de datos (solo activo en entrenamiento)
    x = data_augmentation(x)

    # Bloque Conv 1
    x = layers.Conv2D(32, (3,3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Bloque Conv 2
    x = layers.Conv2D(64, (3,3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Bloque Conv 3
    x = layers.Conv2D(128, (3,3), padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D((2,2))(x)

    # Red densa final
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(dropout_rate)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs, name='flower_cnn')
    return model

model = build_cnn(input_shape=(128,128,3), num_classes=num_classes)
model.summary()


## 7) Modelo — **detalle ampliado** (capas, formas intermedias y parámetros aproximados)

> **Input**: `(None, 128, 128, 3)` — `None` = batch size variable.

Paso a paso (con formas **aproximadas**):

1. **Rescaling(1./255)**

   * Salida: `(None, 128, 128, 3)`
   * Parámetros: 0

2. **data_augmentation(...)**

   * Salida: `(None, 128, 128, 3)`
   * Parámetros: 0 (operaciones sin pesos aprendibles)

3. **Conv2D(32, 3x3, padding='same', activation='relu')**

   * Salida: `(None, 128, 128, 32)`
   * Parámetros: `(3*3*3)*32 + 32 = 896`

     * fórmula: `kernel_h * kernel_w * in_channels * filters + filters(bias)`

4. **BatchNormalization**

   * Salida: `(None, 128, 128, 32)`
   * Parámetros (aprendibles): `2 * 32 = 64` (gamma + beta)

5. **MaxPooling2D(2x2)**

   * Salida: `(None, 64, 64, 32)`
   * Parámetros: 0

6. **Conv2D(64, 3x3, relu)**

   * Salida: `(None, 64, 64, 64)`
   * Parámetros: `(3*3*32)*64 + 64 = 18,496`

7. **BatchNormalization**

   * Parámetros: `2 * 64 = 128`

8. **MaxPooling2D**

   * Salida: `(None, 32, 32, 64)`

9. **Conv2D(128, 3x3, relu)**

   * Salida: `(None, 32, 32, 128)`
   * Parámetros: `(3*3*64)*128 + 128 = 73,856`

10. **BatchNormalization**

    * Parámetros: `2 * 128 = 256`

11. **MaxPooling2D**

    * Salida: `(None, 16, 16, 128)`

12. **GlobalAveragePooling2D**

    * Convierte `(16,16,128)` → `(128)` (promedia cada mapa de características)
    * Parámetros: 0

13. **Dropout(dropout_rate)**

    * No tiene parámetros. Reduce co-adaptaciones.

14. **Dense(128, activation='relu')**

    * Entrada = 128 (salida del GAP)
    * Parámetros: `128*128 + 128 = 16,512`

15. **Dropout(0.3)**

    * Sin parámetros.

16. **Dense(num_classes=5, activation='softmax')**

    * Parámetros: `128*5 + 5 = 645`

**Suma aproximada de parámetros entrenables:** ~**110,853**
(El `model.summary()` en tu ejecución mostrará el total exacto; los cálculos arriba ilustran cómo se obtienen.)

**Motivos para elegir GlobalAveragePooling en vez de Flatten:**

* `GAP` reduce drásticamente la cantidad de parámetros y el riesgo de overfitting.
* Preserva la relación por canal (cada filtro → una entrada).
* Recomendado para tareas de clasificación cuando dejamos que las últimas conv extraigan las características.

**Sobre `BatchNormalization`:**

* Acelera convergencia y permite utilizar tasas de aprendizaje mayores.
* Reduce la sensibilidad a la inicialización.

**Sobre `Dropout`:**

* Apaga aleatoriamente unidades en entrenamiento; útil para regularizar en redes densas.

---


## 8) Compilación y callbacks

**Código asociado:** `model.compile(...)` y lista `callbacks = [...]`

**Qué hace / por qué:**

* `optimizer=Adam(learning_rate=1e-3)`: optimizador adaptativo, buen punto de partida.
* `loss=SparseCategoricalCrossentropy()`: correcta cuando `label_mode='int'` (no hace falta one-hot).
* `metrics=['accuracy']`: métrica principal para clasificación.
* **Callbacks:**

  * `ModelCheckpoint("best_flower_model.h5", save_best_only=True, monitor='val_loss')`: guarda el mejor modelo según `val_loss`. Útil para restaurar la mejor versión.
  * `EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)`: detiene el entrenamiento si `val_loss` no mejora después de `patience` épocas y restaura los pesos mejores.
  * `ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)`: reduce la tasa de aprendizaje si el rendimiento se estanca.

---

In [ ]:

# Compilación del modelo
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

callbacks = [
    ModelCheckpoint("best_flower_model.h5", save_best_only=True, monitor='val_loss'),
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
]

EPOCHS = 30
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=callbacks
)


Gráficos de pérdida y precisión (train vs validation)

In [ ]:
# Gráficos de pérdida y precisión (train vs validation)
import matplotlib.pyplot as plt

def plot_history(history):
    hist = history.history
    epochs = range(1, len(hist['loss'])+1)

    plt.figure(figsize=(14,5))
    plt.subplot(1,2,1)
    plt.plot(epochs, hist['loss'], label='train_loss')
    plt.plot(epochs, hist['val_loss'], label='val_loss')
    plt.title('Pérdida')
    plt.xlabel('Época')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(epochs, hist['accuracy'], label='train_acc')
    plt.plot(epochs, hist['val_accuracy'], label='val_acc')
    plt.title('Precisión')
    plt.xlabel('Época')
    plt.legend()
    plt.show()

plot_history(history)


In [ ]:
# ##### Evaluación final en test set
test_loss, test_acc = model.evaluate(test_ds)
print(f"Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}")

In [ ]:

#Matriz de confusión y reporte por clase

import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

# Obtener etiquetas verdaderas y predichas
y_true = []
y_pred = []
for images, labels in test_ds:
    preds = model.predict(images)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(preds, axis=1))

y_true = np.array(y_true)
y_pred = np.array(y_pred)

cm = confusion_matrix(y_true, y_pred)
print("Classification report:\n", classification_report(y_true, y_pred, target_names=class_names))
print("Confusion matrix:\n", cm)

# Mostrar matriz como imagen
plt.figure(figsize=(8,6))
plt.imshow(cm, interpolation='nearest')
plt.title('Matriz de confusión')
plt.colorbar()
plt.xticks(range(len(class_names)), class_names, rotation=45)
plt.yticks(range(len(class_names)), class_names)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf

def predict_image(model, img_path, class_names, image_size=(128,128)):
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    x = x.astype('float32') / 255.0
    x = np.expand_dims(x, axis=0)
    preds = model.predict(x)[0]
    top_idx = np.argmax(preds)
    return top_idx, preds[top_idx], preds

# Uso:
idx, prob, all_probs = predict_image(model, "probar.jpg", class_names)
print(class_names[idx], prob)



In [ ]:
model.save("flower_model.h5")
# cargar
from tensorflow.keras.models import load_model
model2 = load_model("flower_model.h5")


In [ ]:
model.save("flower_savedmodel")   # carpeta con formato SavedModel
# cargar
model3 = tf.keras.models.load_model("flower_savedmodel")
